[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_portfolio/EMQA_portfolio.ipynb)

# EMQA_portfolio
Portfolio optimization and efficient frontier for energy commodities.
**Output:** `portfolio_frontier.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
# Fetch 4 energy tickers
tickers = {'Brent': 'BZ=F', 'WTI': 'CL=F', 'NatGas': 'NG=F', 'HeatingOil': 'HO=F'}
prices = pd.DataFrame()
for label, tk in tickers.items():
    try:
        s = fetch(tk)
        prices[label] = s
    except Exception as e:
        print(f"Warning: {tk} failed ({e})")

prices = prices.dropna()
returns = np.log(prices / prices.shift(1)).dropna()
print(f"Assets: {list(returns.columns)}, Observations: {len(returns)}")

mu_annual = returns.mean() * 252
cov_annual = returns.cov() * 252
print("\nAnnualized returns:")
print(mu_annual.round(4))

In [ ]:
# Monte Carlo portfolios
np.random.seed(42)
n_portfolios = 10000
n_assets = len(returns.columns)

results = np.zeros((n_portfolios, 3))
weights_record = np.zeros((n_portfolios, n_assets))

for i in range(n_portfolios):
    w = np.random.dirichlet(np.ones(n_assets))
    weights_record[i] = w
    port_ret = np.dot(w, mu_annual.values)
    port_vol = np.sqrt(np.dot(w.T, np.dot(cov_annual.values, w)))
    sharpe = port_ret / port_vol if port_vol > 0 else 0
    results[i] = [port_vol, port_ret, sharpe]

# Min variance and max Sharpe
idx_min_var = results[:, 0].argmin()
idx_max_sharpe = results[:, 2].argmax()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

# Scatter all portfolios
sc = ax.scatter(results[:, 0] * 100, results[:, 1] * 100, c=results[:, 2],
                cmap='RdYlGn', s=5, alpha=0.5)
plt.colorbar(sc, ax=ax, label='Sharpe Ratio', shrink=0.8)

# Mark min-variance
ax.scatter(results[idx_min_var, 0] * 100, results[idx_min_var, 1] * 100,
           color=COLORS['blue'], marker='*', s=300, zorder=5, edgecolors='black',
           linewidths=0.8, label='Min Variance')

# Mark max-Sharpe
ax.scatter(results[idx_max_sharpe, 0] * 100, results[idx_max_sharpe, 1] * 100,
           color=COLORS['red'], marker='*', s=300, zorder=5, edgecolors='black',
           linewidths=0.8, label='Max Sharpe')

# Mark individual assets
asset_colors = [COLORS['blue'], COLORS['green'], COLORS['orange'], COLORS['purple']]
for j, col in enumerate(returns.columns):
    w_single = np.zeros(n_assets)
    w_single[j] = 1.0
    ret_j = mu_annual.values[j] * 100
    vol_j = np.sqrt(cov_annual.values[j, j]) * 100
    ax.scatter(vol_j, ret_j, color=asset_colors[j % len(asset_colors)],
               marker='D', s=100, zorder=6, edgecolors='black', linewidths=0.8, label=col)

ax.set_xlabel('Annualized Volatility (%)')
ax.set_ylabel('Annualized Return (%)')
ax.set_title('Energy Portfolio - Efficient Frontier (Monte Carlo)')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=3)

plt.tight_layout()
save_fig(fig, 'portfolio_frontier.pdf')
plt.show()